# **Multi-classification problems occur when we have a class with more than two classes**

NOTE: 
* In chapter 2, we dealt with the development of a classifier for a two-class problem. There the values of the class were provided for us in numerical form. 
* In this chapter:
    - we shall deal with a problem where the class is more than 2 (this is called multi-class problem)
    - Furthermore, the data provided for this problem is such that the class are provided in "string" form rather than numerical form required by most neural network framework.
    - Based on the point above, we shall be using the Categorical Crossentropy as the loss function.
        - **Categorical Crossentropy** measures the difference between the predicted probabilities and the true label of the class we should have predicted. With this loss function, the loss is always big if the predicted probabilites is different from what we should have predicted. Deep dip on this. 
        - **Together with this, we also introduced another activation function: the **softmax******.

* For this problem, the output will be vectors containing the probabilities of each class. *However, the output in the dataset are in the form of strings, so they must first be converted into categorical variables*

* This less will show, how to first treat string output by converting them into categorical variable and then follow by one-hot encoding of this new variable


## Problem statement:
*We have data about the positions of dart throws by four individuals. Using the position coordinates of past throws by these competitors, we wish to build a classifier that can differentiates who throws the dart. This problem is a multi-class classification problem since each dart can only be thrown by one of 4 competitors. So classes/labels are mutually exclusive, and therefore we can build classifier that will have a final layer that will have as many neurons as the competitors, pass the output of this layer to the softmax activation function to achieve a total sum of probabilities of 1 over all competitors.*

In [1]:
import pandas as pd
import numpy as np
from keras.api.models import Sequential
from keras.api.layers import Dense, InputLayer
from keras.api.utils import to_categorical, set_random_seed

from sklearn.model_selection import train_test_split

set_random_seed(123)


## **Data importation and assessment**


In [2]:
# Import the data
dart = pd.read_csv("/kaggle/input/dart-throws/darts_throws_competitors.csv")

# Check for data info - which will tell you the datatype and count
print(dart.info())

# Check for missing values
print(dart.isnull().sum())

# Print out a few rows of data to check
print(dart.head(4))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   xCoord      800 non-null    float64
 1   yCoord      800 non-null    float64
 2   competitor  800 non-null    object 
dtypes: float64(2), object(1)
memory usage: 18.9+ KB
None
xCoord        0
yCoord        0
competitor    0
dtype: int64
     xCoord    yCoord competitor
0  0.196451 -0.520341      Steve
1  0.476027 -0.306763      Susan
2  0.003175 -0.980736    Michael
3  0.294078  0.267566       Kate


## **Data pre-processing**
    - Using the pandas Categorical() function here to convert string object to categorical object

In [3]:
# As you can see from above, the competitor column is a string

# Separate out the input and output data
input_dat = dart.drop(["competitor"], axis = 1).values
output = dart["competitor"]

# Convert output to categorical using pandas Categorical() function
output_cat = pd.Categorical(output)                  ## Converts ordinary string to categorical object
output_cat_numbers = output_cat.codes                ## This will encode the labels with number

# One hot encoding the above categorical numbers
output_cat_encoded = to_categorical(output_cat_numbers) ## This converts a vector into matrix


print(output_cat[:8])
# Now print the one-hot encoded labels
print('One-hot encoded competitors: \n',output_cat_encoded[:8])



['Steve', 'Susan', 'Michael', 'Kate', 'Steve', 'Kate', 'Kate', 'Steve']
Categories (4, object): ['Kate', 'Michael', 'Steve', 'Susan']
One-hot encoded competitors: 
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


### This is an alternative approach for the conversion and encoding

In [7]:
## An alternative method to convert the string object to numerical variable is via factors
output_factorized = output.copy().factorize()[0]
#print(output_factorized)

# One hot encoding the above categorical numbers
output_factorized_encoded = to_categorical(output_factorized) ## This converts a vector into matrix
print(output_factorized_encoded[:8])

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


## **Split the data into training and testing components**

In [4]:
X_train, X_test, y_train, y_test = train_test_split(input_dat, output_cat_encoded,
                                                   test_size=0.2, stratify=output_cat_encoded)

## **Build the model architecture**
    - We wish to build a model with 3 dense layers of 128, 64 and 32 neurons each.
    - Take note that we employ the softmax activation function here.

In [5]:
model = Sequential()
model.add(InputLayer(shape = (2,)))
model.add(Dense(128, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))

# For the output layer below,we add a dense layer with as many neurons as competitors: 4
# We are also using the "softmax" activation function
model.add(Dense(4, activation = "softmax"))

model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,852 (42.39 KB)

 Trainable params: 10,852 (42.39 KB)

 Non-trainable params: 0 (0.00 B)

Softmax:

 - The output of a  vector of numbers pass to the softmax function is always between 0 and 1, and the sum of the outputs is always 1. It has the following formula: 
    $$ softmax(x_i) = e^{x_i}/\sum_j {e^{x_j}}$$

    - Use case: Used for multi-class classification, where the output can be interpreted as the probabilities of different classes.
    - Simply, softmax converts the raw class scores into a probability distribution, ensuring that the total sum of probabilities across all classes is 1.

## **Compliling the model**

In [6]:
# Compile your model using categorical_crossentropy loss
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

## **Fit the model on training and testing data**

In [7]:
model_fit = model.fit(X_train, y_train, epochs = 200, verbose = False)


In [36]:
print(model_fit)
#print(model_fit.history["accuracy"])

## **Evaluate the model**

In [8]:
model_eval = model.evaluate(X_test, y_test)
print(model_eval[1])

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7989 - loss: 0.6411  
0.793749988079071


**NOTE**: *When computing the accuracy above, our model takes the class with the highest probability as the prediction. To verify this, we shall unpack the predictions using the .predict() function.*

## **Softmax predictions**
    - The predict function always returns numpy array

In [9]:
preds = model.predict(X_test)
print(preds[:5])                               ## Print only the first 5 predictions



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[0.24896052 0.25753847 0.00905691 0.4844441 ]
 [0.18438223 0.4074901  0.00924431 0.3988834 ]
 [0.9539735  0.01862632 0.01015766 0.01724253]
 [0.62357306 0.01534733 0.02088005 0.34019953]
 [0.10753637 0.00475325 0.00857344 0.879137  ]]


**NOTE**: 
> As you can observe above, the prediction we get directly from the last. This is because we used the softmax activation function. As a result, for every input of 2 coordinates provided to our model in the X_test data, there's an output vector of 4 numbers. Each of these numbers encodes the probability of a given dart being thrown by one of the 4 possible competitors. Premised on this, you must know that when computing accuracy with the .evaluate() method, our model takes the class with the highest probability as the prediction. 

We can do similar thing via the np.argmax() as shown below.

In [11]:
# A function that outputs the maximum within the 4 outputs of the last sigmoid layer

# A useful numpy command to use here is np.argmax() function that returns the position of max in an array
postion_max_probab = [np.argmax(k) for k in preds]
position_max_encoded = [np.argmax(k) for k in y_test]
dict_predict_truelabel = {"Predictions":postion_max_probab, "True labels":position_max_encoded}
df = pd.DataFrame(dict_predict_truelabel)
print(df.head(15))

# for i,pred in enumerate(preds):
#     print("{} | {}".format(np.argmax(pred), np.argmax(y_test[i])))

    Predictions  True labels
0             3            2
1             1            3
2             0            0
3             0            1
4             3            3
5             2            2
6             1            1
7             2            2
8             3            3
9             2            2
10            1            1
11            1            1
12            3            3
13            0            0
14            3            3
